# Fra guiden i INF265

In [6]:
from matplotlib import pyplot as plt
import numpy as np
import torch
from torchvision import datasets, transforms
from collections import Counter
from torch.utils.data import random_split

torch.manual_seed(123)

In [7]:
data_path = '/cifar-to-patches-py'

cifar10_train_val = datasets.CIFAR10(
    data_path,
    train=True,
    download=True,
)
cifar10_test = datasets.CIFAR10(data_path, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
n_val = 5000
n_train = len(cifar10_train_val)-n_val


cifar10_train, cifar10_val = random_split(
    cifar10_train_val,
    [n_train, n_val],
    generator=torch.Generator().manual_seed(123)
)

print("Size of the train dataset:        ", len(cifar10_train))
print("Size of the validation dataset:   ", len(cifar10_val))
print("Size of the test dataset:         ", len(cifar10_test))

Size of the train dataset:         45000
Size of the validation dataset:    5000
Size of the test dataset:          10000
